## Introduction:
GAN i.e. generative adversarial network was first introduced into the world by Ian Goodfellow in 2014. It is the type of neural network used to generate new images. In this notebook, we will explore GAN networks, and train a GAN to generate few artificial images ourselves.<br/>
This notebook is forked from [mehmet lauda tekman's code](https://www.kaggle.com/mehmetlaudatekman/dcgans-face-generation).<br/>
There are a quite a number of different GAN architectures like [vanilla GAN](https://www.kaggle.com/zorroblue/vanilla-gan-on-fashion-mnist), DCGAN i.e. deep convolutional GAN etc. In this notebook, we are implementing DCGAN.<br/>
Here are a few resources which helped develop this notebook as well as to understand GAN training.<br/>
(1)[celeb dataset original link](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)<br/>

(3)[code inspiration](https://towardsdatascience.com/image-generation-in-10-minutes-with-generative-adversarial-networks-c2afc56bfa3b)<br/>
(4)[random sampling from numpy array](https://www.kite.com/python/answers/how-to-select-random-rows-from-a-numpy-array-in-python)<br/>
(5)[How to identify gan training failures](https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/)<br/>

In [ ]:
import numpy as np
import PIL
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import random

MAIN_PATH = "../input/celeba-dataset/img_align_celeba/img_align_celeba"


* First, we are going to create a list that contains paths of images. We'll get random paths from that list.

In [ ]:
image_paths = glob(MAIN_PATH+"/*")


In [ ]:
len(image_paths)

* We have 202K images in CelebA dataset, so we won't read all images, we'll read them as batches.

Let's define a function. This function will take path argument read image from that path.

In [ ]:
def readImage(path,image_size=(256,256)):
    img = np.asarray(PIL.Image.open(path).resize(image_size))
    # img - 127.5 / 127.5 ==> compress between [-1,1]
    img = ((img - 127.5) / 127.5).astype("float32")
    return img

* Let's test our function.

In [ ]:
test_img = readImage("../input/celeba-dataset/img_align_celeba/img_align_celeba/000007.jpg")
print(test_img.shape)

plt.imshow(test_img)
plt.show()

* I said we'll read images as batches so now we'll define a generator that yield one batch data.

In [ ]:
BATCH_SIZE = 128
#STEPS_PER_EPOCH = len(image_paths) // BATCH_SIZE
#1582 steps are too many steps. Let's just train for 500 steps.
STEPS_PER_EPOCH = 500
print("Steps per epochh are",STEPS_PER_EPOCH)
def dataGenerator(batch_size):
    while True:
        paths = random.choices(image_paths,k=batch_size)
        batch = []
        for p in paths:
            batch.append(readImage(p))
        
        yield np.asarray(batch)

dataGen = dataGenerator(BATCH_SIZE)
print(next(dataGen).shape)


* Our data is ready, let's start to build our GANs

First, we'll create our Generator. Generator will upsample our seed (I explained it in introduction) using convolutional transpose layers (upsampling layers)

In [ ]:
WEIGHT_INIT = tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.2)
def make_generator():
    model = tf.keras.Sequential()
    
    # Random noise to 16x16x256 image
    model.add(layers.Dense(16*16*256,use_bias=False,input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((16,16,256)))
    
    assert model.output_shape == (None,16,16,256)
    
    model.add(layers.Conv2DTranspose(128,(5,5),strides=(2,2),use_bias=False,padding="same",kernel_initializer=WEIGHT_INIT))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    assert model.output_shape == (None,32,32,128)
    
    model.add(layers.Conv2DTranspose(128,(5,5),strides=(2,2),use_bias=False,padding="same",kernel_initializer=WEIGHT_INIT))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    assert model.output_shape == (None,64,64,128)
    
    model.add(layers.Conv2DTranspose(64,(5,5),strides=(2,2),use_bias=False,padding="same",kernel_initializer=WEIGHT_INIT))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    assert model.output_shape == (None,128,128,64)
    
    model.add(layers.Conv2DTranspose(3,(5,5),strides=(2,2),use_bias=False,padding="same",kernel_initializer=WEIGHT_INIT,
                                     activation="tanh"
                                    ))
              # Tanh activation function compress values between -1 and 1. 
              # This is why we compressed our images between -1 and 1 in readImage function.
    assert model.output_shape == (None,256,256,3)
    return model

In [ ]:
generator = make_generator()

In [ ]:
generator.summary()

In [ ]:
# we'll use cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    # First argument of loss is real labels
    # We've labeled our images as 1 (real) because
    # we're trying to fool discriminator
    return cross_entropy(tf.ones_like(fake_output),fake_output)

gen_optimizer = tf.keras.optimizers.Adam(lr=1e-4)


* Now we'll build our discriminator network.
* Discriminator network is a CNN based image classifier that classify images real or fake.

In [ ]:
def make_discriminator():
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(64,(5,5),strides=(2,2),padding="same",input_shape=(256,256,3)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128,(5,5),strides=(2,2),padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(265,(5,5),strides=(2,2),padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

In [ ]:
discriminator = make_discriminator()


In [ ]:
def discriminator_loss(real_images,fake_images):
    real_loss = cross_entropy(tf.ones_like(real_images),real_images)
    fake_loss = cross_entropy(tf.zeros_like(fake_images),fake_images)
    total_loss = real_loss + fake_loss
    return total_loss


In [ ]:
discriminator_optimizer = tf.keras.optimizers.Adam(lr=1e-4)

* Now we'll train model. 

In [ ]:
EPOCHS = 5
NOISE_DIM = 100

@tf.function
def train_step(images):
    # We've created random seeds
    noise = tf.random.normal([BATCH_SIZE,NOISE_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generator generated images
        generated_images = generator(noise,training=True)
        
        # We've sent our real and fake images to the discriminator
        # and taken the decisions of it.
        real_output = discriminator(images,training=True)
        fake_output = discriminator(generated_images,training=True)
        
        # We've computed losses of generator and discriminator
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)
    
    # We've computed gradients of networks and updated variables using those gradients.
    gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    gen_optimizer.apply_gradients(zip(gradients_of_generator,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))
        

In [ ]:
import time
import sys
def train(epochs):
    for epoch in range(epochs):
        start = time.time()
        for step in range(STEPS_PER_EPOCH):
            train_step(next(dataGen))
    
            sys.stdout.write(f"\rSTEP: {step}/{STEPS_PER_EPOCH}")
            sys.stdout.flush()
            
        finish_time = round(time.time() - start,2)
        print(f"Epoch {epoch}/{epochs} Process Time : {finish_time}")
        print("-"*15)
        

In [ ]:
train(EPOCHS)

In [ ]:
train(3)

* Let's generate and display some synthetic images.

In [ ]:
noise = tf.random.normal([16,100])
generated_images = np.asarray(generator(noise,training=False))

fig = plt.figure(figsize=(6,6))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow((generated_images[i,:,:,:]*127.5+127.5).astype("int"))
    plt.axis("off")
    
plt.show()

## save the model for future use
[Related resource](https://www.tensorflow.org/tutorials/keras/save_and_load)

In [ ]:
generator.save('/kaggle/working/shyam_face_generator')

## Conclusion:
As you can see, some amount of human like faces are coming up. This is the basic DCGAN implementation. Obviously, increasing the epoch numbers can improve the quality of the images. Hope you liked the content. Please comment and share your views about the project.<br/> For limited kernel time(9hrs), we have only been able to create low class images. You can try and train for longer time, and get better images. i.e. Increasing the epoch will improve the quality.
So, that's it folks! hope you learnt some and liked it!